In [32]:
import os
import pandas as pd
from pathlib import Path
import networkx as nx

from config.settings import *

STRING_DIR = DATA_DIR / "prior_knowledge_network_data" / "mm10" / "STRING"
TRRUST_DIR = DATA_DIR / "prior_knowledge_network_data" / "mm10" / "TRRUST"
KEGG_DIR = DATA_DIR / "prior_knowledge_network_data" / "mm10" / "KEGG"

string_pkn_file = STRING_DIR / "string_mm10_pkn.csv"
trrust_pkn_file = TRRUST_DIR / "trrust_mm10_pkn.csv"
kegg_pkn_file = KEGG_DIR / "kegg_mm10_pkn.csv"

peak_gene_dist_df = pd.read_parquet("data/processed/mESC/E7.5_rep1/peak_to_gene_dist.parquet")
sliding_window_df = pd.read_parquet("data/processed/mESC/E7.5_rep1/sliding_window.parquet")
tf_tg_reg_pot_df = pd.read_parquet("data/processed/mESC/E7.5_rep1/tf_tg_regulatory_potential.parquet")
tf_tg_combos_w_attr_df = pd.read_parquet("data/processed/mESC/E7.5_rep1/tf_tg_combos_with_attributes.parquet")
tf_tg_combos_w_pkn_df = pd.read_parquet("data/processed/mESC/E7.5_rep2/tf_tg_data.parquet")

tf_list = pd.read_csv("data/processed/mESC/tf_tg_combos/tf_list.csv")


In [33]:
print("peak_gene_dist_df")
print(peak_gene_dist_df.head())
print()
print("sliding_window_df")
print(sliding_window_df.head())
print()
print("tf_tg_reg_pot_df")
print(tf_tg_reg_pot_df.head())
print()
print("tf_tg_combos_w_attr_df")
print(tf_tg_combos_w_attr_df.head())
print()
print("tf_tg_combos_w_pkn_df")
print(tf_tg_combos_w_pkn_df.head())
print("tf_list")
print(tf_list.head())

peak_gene_dist_df
        peak_chr  peak_start   peak_end                    peak_id gene_chr  \
1772072     chr9    96895014   96895614     chr9:96895014-96895614     chr9   
1360760     chr5   146230649  146231249   chr5:146230649-146231249     chr5   
1307851     chr5   113772200  113772800   chr5:113772200-113772800     chr5   
576060     chr15    76080271   76080871    chr15:76080271-76080871    chr15   
350656     chr11   116130529  116131129  chr11:116130529-116131129    chr11   

         gene_start   gene_end target_id  TSS_dist  TSS_dist_score  
1772072    96895614   96895614      PPIA         0         1.00000  
1360760   146231249  146231249      CDK8         0         1.00000  
1307851   113772800  113772800      ISCU         0         1.00000  
576060     76080870   76080870     PUF60         1         0.99996  
350656    116131128  116131128    TRIM65         1         0.99996  

sliding_window_df
                peak_id   TF  sliding_window_score
0  chr1:3142536-3143136

In [19]:
string_pkn_df = pd.read_csv(string_pkn_file)
trrust_pkn_df = pd.read_csv(trrust_pkn_file)
kegg_pkn_df = pd.read_csv(kegg_pkn_file)

# string_pkn_df["TF"] = string_pkn_df["TF"].str.upper()
# string_pkn_df["TG"] = string_pkn_df["TG"].str.upper()

# trrust_pkn_df["TF"] = trrust_pkn_df["TF"].str.upper()
# trrust_pkn_df["TG"] = trrust_pkn_df["TG"].str.upper()

# kegg_pkn_df["TF"] = kegg_pkn_df["TF"].str.upper()
# kegg_pkn_df["TG"] = kegg_pkn_df["TG"].str.upper()

In [20]:
print(tf_tg_data_df.head())
print(tf_tg_data_df["TF"].unique().tolist()[:10])
print(tf_tg_data_df["TG"].unique().tolist()[:10])


    TF             TG  reg_potential  motif_density  mean_tf_expr  \
0  AHR  0610009E02RIK       6.143358       4.521789       0.02268   
1  AHR  0610010F05RIK       0.746198       3.988984       0.02268   
2  AHR  0610040J01RIK       0.050996       4.007333       0.02268   
3  AHR  1010001N08RIK       0.000000       0.000000       0.02268   
4  AHR  1110046J04RIK       0.063120       3.931826       0.02268   

   mean_tg_expr  expr_product  log_reg_pot  motif_present  
0      0.015923      0.000361     1.966183              1  
1      0.069961      0.001587     0.557441              1  
2      0.065866      0.001494     0.049738              1  
3      0.018050      0.000409     0.000000              0  
4      0.012723      0.000289     0.061208              1  
['AHR', 'AHRR', 'ALX1', 'AR', 'ARID5B', 'ARNT2', 'BACH2', 'BATF2', 'BAZ2B', 'BCL11A']
['0610009E02RIK', '0610010F05RIK', '0610040J01RIK', '1010001N08RIK', '1110046J04RIK', '1110051M20RIK', '1500015A07RIK', '1600002D24RIK', '1

In [21]:
print("STRING")
print(string_pkn_df.head())
print("\nTRRUST")
print(trrust_pkn_df.head())
print("\nKEGG")
print(kegg_pkn_df.head())

STRING
      TF     TG  string_neighborhood_score  string_fusion_score  \
0  Gnai3   Rgs4                          0                    0   
1  Gnai3   Drd2                          0                    0   
2  Gnai3   Gnb4                          0                    0   
3  Gnai3   Rgs3                          0                    0   
4  Gnai3  Gnai1                          0                    0   

   string_cooccurence_score  string_coexpression_score  \
0                         0                         56   
1                         0                          0   
2                         0                        151   
3                         0                         90   
4                        47                          0   

   string_experimental_score  string_database_score  string_textmining_score  \
0                        594                    500                      492   
1                        604                    900                      301   
2

In [22]:
pkn_edges = set(zip(string_pkn_df["TF"], string_pkn_df["TG"])) | set(zip(trrust_pkn_df["TF"], trrust_pkn_df["TG"])) | set(zip(kegg_pkn_df["TF"], kegg_pkn_df["TG"]))

print(f"Number of edges in PKN: {len(pkn_edges)}")

Number of edges in PKN: 345292


In [23]:
print(list(pkn_edges)[:5])

[('Bub1b', 'Bub3'), ('Gm20806', 'Srsy'), ('Ehhadh', 'Echdc1'), ('CCL2', 'CCR4'), ('Taf4', 'Sgf29')]


In [24]:
print(f"Overlapping TFs: {len(set(tf_tg_data_df['TF']).intersection({t for t,_ in pkn_edges}))}")
print(f"Overlapping TGs: {len(set(tf_tg_data_df['TG']).intersection({t for _,t in pkn_edges}))}")

Overlapping TFs: 181
Overlapping TGs: 845


In [26]:
from typing import Tuple
def select_pkn_edges_from_df(df: pd.DataFrame, pkn_edges: set[Tuple[str, str]]):
    df['TF'] = df['TF'].str.upper()
    df['TG'] = df['TG'].str.upper()
    
    df['in_pkn'] = df.apply(
        lambda r: int((r['TF'], r['TG']) in pkn_edges or (r['TG'], r['TF']) in pkn_edges),
        axis=1
    )
    
    in_pkn_df = df[df['in_pkn'] == 1]
    not_in_pkn_df = df[df['in_pkn'] == 0]
    
    return in_pkn_df, not_in_pkn_df
    
in_pkn_df, not_in_pkn_df = select_pkn_edges_from_df(tf_tg_data_df, pkn_edges)

In [27]:
print(f"Number of edges in TF-TG data: {tf_tg_data_df.shape[0]:,}")
print(f"Number of edges in PKN: {len(pkn_edges):,}")
print()
print(f"Number of unique TFs not in PKN: {not_in_pkn_df['TF'].nunique():,}")
print(f"Number of unique TGs not in PKN: {not_in_pkn_df['TG'].nunique():,}")
print()
print(f"Number of edges in TF-TG data that are not in PKN: {not_in_pkn_df.shape[0]:,}")
print(f"Number of edges in TF-TG data that are in PKN: {in_pkn_df.shape[0]:,}")
print()
print(f"Fraction of edges in TF-TG data that are in PKN: {in_pkn_df.shape[0] / tf_tg_data_df.shape[0]:.2f}")

Number of edges in TF-TG data: 680,595
Number of edges in PKN: 345,292

Number of unique TFs not in PKN: 255
Number of unique TGs not in PKN: 2,669

Number of edges in TF-TG data that are not in PKN: 679,859
Number of edges in TF-TG data that are in PKN: 736

Fraction of edges in TF-TG data that are in PKN: 0.00


In [14]:
in_pkn_df.columns

Index(['TF', 'TG', 'reg_potential', 'motif_density', 'mean_tf_expr',
       'mean_tg_expr', 'expr_product', 'log_reg_pot', 'motif_present',
       'in_pkn'],
      dtype='object')

In [18]:
# Sample an equal number of negatives
neg_df = not_in_pkn_df.sample(n=len(in_pkn_df), random_state=42)

balanced_df = pd.concat([in_pkn_df, neg_df], ignore_index=True)
print(f"Balanced fine-tuning set: {len(in_pkn_df)} positives, {len(neg_df)} negatives")


Balanced fine-tuning set: 9867 positives, 9867 negatives


In [19]:
merged_df = balanced_df.merge(string_pkn_df, on=["TF", "TG"], how="left")
merged_df = merged_df.merge(trrust_pkn_df, on=["TF", "TG"], how="left")
merged_df = merged_df.merge(kegg_pkn_df, on=["TF", "TG"], how="left")

merged_df.head()

,TF,TG,reg_potential,motif_density,mean_tf_expr,mean_tg_expr,expr_product,log_reg_pot,motif_present,in_pkn,...,string_database_score,string_textmining_score,string_combined_score,trrust_sign,trrust_regulation,trrust_pmids,trrust_support_n,kegg_signal,kegg_n_pathways,kegg_pathways
0,AHR,MFSD2A,0.0,0.0,0.071921,0.000108,0.000008,0.0,0,1,...,NaN,NaN,NaN,0.0,Unknown,21736709,1.0,NaN,NaN,NaN
1,AHR,EPHX4,0.0,0.0,0.071921,0.000029,0.000002,0.0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,hsa05207
2,AHR,GSTM4,0.0,0.0,0.071921,0.000162,0.000012,0.0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,hsa05207
3,AHR,GSTM2,0.0,0.0,0.071921,0.000095,0.000007,0.0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,hsa05207
4,AHR,GSTM1,0.0,0.0,0.071921,0.000022,0.000002,0.0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,hsa05207


In [20]:
print(f"Number of edges in merged dataframe: {merged_df.shape[0]:,}")
print(f"Number of unique TFs in merged dataframe: {merged_df['TF'].nunique():,}")
print(f"Number of unique TGs in merged dataframe: {merged_df['TG'].nunique():,}")
print(f"Number of unique TF-TG pairs in merged dataframe: {merged_df[['TF', 'TG']].drop_duplicates().shape[0]:,}")
print(merged_df.head())


Number of edges in merged dataframe: 19,734
Number of unique TFs in merged dataframe: 240
Number of unique TGs in merged dataframe: 10,319
Number of unique TF-TG pairs in merged dataframe: 19,734
    TF      TG  reg_potential  motif_density  mean_tf_expr  mean_tg_expr  \
0  AHR  MFSD2A            0.0            0.0      0.071921      0.000108   
1  AHR   EPHX4            0.0            0.0      0.071921      0.000029   
2  AHR   GSTM4            0.0            0.0      0.071921      0.000162   
3  AHR   GSTM2            0.0            0.0      0.071921      0.000095   
4  AHR   GSTM1            0.0            0.0      0.071921      0.000022   

   expr_product  log_reg_pot  motif_present  in_pkn  ...  \
0      0.000008          0.0              0       1  ...   
1      0.000002          0.0              0       1  ...   
2      0.000012          0.0              0       1  ...   
3      0.000007          0.0              0       1  ...   
4      0.000002          0.0              0    

In [21]:
merged_df.to_parquet("data/processed/PBMC/LINGER_PBMC_SC_DATA/tf_tg_merged_features.parquet")